In [ ]:
import pandas as pd
import random

In [ ]:
import requests
import json
from gen import *

In [3]:
URL = "http://localhost:8080"

In [4]:
def auth_dhis(url):
    url = url+ "/api/me"
    session = requests.Session()
    session.auth = ('admin', 'district')
    return session

In [5]:
def get_data(key):
    url = URL + "/api/me"
    session = requests.Session()
    session.auth = ('admin', 'district')
    response = session.get(URL + "/api/" + key )
    if response.status_code == 200:
        json_response = json.loads(response.text)
        return json_response
    else:
        print("bad response: " + str(response.status_code))
        return None

In [ ]:
response = get_data("organisationUnits") # json


In [197]:
# response = session.delete(URL + "/organisationUnits/" + "SRW3lubkRqd")
# NO FUNCIONA! 
response = session.patch(URL + "/api/maintenance/dataPruning/organisationUnits/" + "SRW3lubkRqd")
print(response.status_code)

405


In [8]:
json_response = get_data("organisationUnits")
orgunits = json_response["organisationUnits"]

In [206]:
# atribute geometry tiene id:"CrJdK4CsRGs" entonces tengo que borrar ese atributo ? 
def delete_OrgUnit_by_parent(parent_name):
    ids = []
    response = get_data("organisationUnits") # json
    if response:
        orgUnits_dic = response["organisationUnits"] #list of dics
    else:
        return None
    orgUnits_df = pd.DataFrame(orgUnits_dic)
    parent = org_units_df[org_units_df["displayName"] == parent_name ].to_dict("records")
    if parent:
        id_parent = parent[0]['id']
    else:
        print("there is no org unit call" + parent_name)
    for i in orgUnits_dic:
        org_unit = get_data("organisationUnits/" + i["id"])
        try:
            t =  org_unit["parent"]["id"]
            if org_unit["parent"]["id"] == id_parent:
                ids.append(i["id"])
                
#                 response = session.delete(URL + "/organisationUnits/"+ i["id"])
#                 if response.status_code == 200:
#                     print("La unidad organizativa" + i["displayName"] +  "se eliminó con éxito.")
#                 else:
#                     print("Hubo un error al eliminar la unidad organizativa. Código de estado HTTP:", response.status_code)
        except KeyError:
            print("this is a root organizationUnit")
    d =  {"organisationUnits": [{"id": id} for id in ids]}
    # TODO una vez hecho esto este diccionario habrá qeu pasarlo po maintanence con la opción delete
    return json.dumps(d)
    
# https://docs.dhis2.org/archive/en/2.28/developer/html/webapi_metadata_import.html            



        

In [209]:
import geopandas as gpd


In [271]:
# Load GeoJSON file
gdf = gpd.read_file('../org_units/mapas/municipios_desde2007_20170101.json')

# Convert GeoDataFrame to DataFrame
df = pd.DataFrame(gdf)

df.head()

,geocode,geopadre,etiqueta,notas,granularidad,gcd_provincia,gcd_isla,gcd_grancomarca,gcd_comarca,ign_sup,ign_perim,utm_x,utm_y,longitud,latitud,utm_x_capi,utm_y_capi,long_capi,lati_capi,geometry
0,35001,ES705A22,Agaete,Ayuntamiento de Agaete,MUNICIPIOS,ES701,ES705,ES705A2,ES705A22,4451.16,41038.0,432237.32,3105495.86,-15.689645,28.073135,431360.28,3108387.71,-15.698739,28.099193,(POLYGON ((-15.74603672847631 28.0510585578892...
1,35002,ES705A32,Agüimes,Ayuntamiento de Agüimes,MUNICIPIOS,ES701,ES705,ES705A3,ES705A32,7889.72,55508.0,455417.29,3085973.99,-15.453004,27.897893,456117.58,3086639.88,-15.445914,27.903927,(POLYGON ((-15.38690327697937 27.9025099133621...
2,35003,ES704A01,Antigua,Ayuntamiento De Antigua,MUNICIPIOS,ES701,ES704,ES704A0,ES704A01,25024.80,79292.0,603336.94,3136732.92,-13.945579,28.352768,596564.74,3144502.79,-14.014024,28.423412,(POLYGON ((-13.91950980860212 28.2514875720348...
3,35004,ES708A01,Arrecife,Ayuntamiento De Arrecife,MUNICIPIOS,ES701,ES708,ES708A0,ES708A01,2427.54,36641.0,641157.78,3206510.34,-13.551117,28.978878,641613.10,3204338.91,-13.546719,28.959235,(POLYGON ((-13.58358501395838 28.9519680854640...
4,35005,ES705A23,Artenara,Ayuntamiento de Artenara,MUNICIPIOS,ES701,ES705,ES705A2,ES705A23,6641.90,50791.0,432859.41,3099392.63,-15.682966,28.018075,436451.97,3099620.39,-15.646436,28.020308,(POLYGON ((-15.74959185013334 27.9880062442141...


In [272]:
def get_org_unit_id(name):
    ids = []
    response = get_data("organisationUnits") # json
    if response:
        orgUnits_dic = response["organisationUnits"] #list of dics
    else:
        return None
    orgUnits_df = pd.DataFrame(orgUnits_dic)
    orgUnit = orgUnits_df[orgUnits_df["displayName"] == name ].to_dict("records")
    if orgUnit:
        orgUnit_id = orgUnit[0]['id']
        return orgUnit_id
    else:
        print("no org unit called " + name)
        return None
    

In [273]:
canarias_id = get_org_unit_id("Canarias")


In [274]:
org_unit_can = pd.DataFrame(
    {
        "name" : df["etiqueta"].to_list()
    }
)

In [275]:
org_unit_can

,name
0,Agaete
1,Agüimes
2,Antigua
3,Arrecife
4,Artenara
...,...
83,Vallehermoso
84,La Victoria de Acentejo
85,Vilaflor de Chasna
86,Villa de Mazo


In [276]:
org_unit_can["uid"] = ""
org_unit_can["code"] = ""
org_unit_can["parent"] = canarias_id
org_unit_can["shortname"] = df["etiqueta"].to_list()


In [287]:
org_unit_can.reset_index(drop=True, inplace=True)

In [289]:
org_unit_can.to_csv("org_unit_can.csv", index = "false") # esto no quita la columna indez 

In [284]:
!ls

fake-data-dhis2.ipynb	main.py			  Untitled.ipynb
gen			mauritania_org_units.txt
IDS_AGG_1.2.0_DHIS2.39	org_unit_can


In [280]:
def import_data_csv(file, key):
    url = URL + "/api/me"
    session = requests.Session()
    session.auth = ('admin', 'district')
    response = session.post(URL + "/api/" + key )
    params = {
        
    }